In [178]:
import pennylane as qml
from pennylane import numpy as np

# 양자 장치 설정
dev = qml.device("default.mixed", wires=2)


def circuit_real(theta):
    qml.RX(theta[0], wires=0)
    qml.CNOT( wires=[0,1])
    qml.RY(theta[1], wires=0)

def circuit(theta,p):
    qml.RX(theta, wires=0)
    qml.DepolarizingChannel(p, wires=0)
    qml.CNOT( wires=[0,1])
    qml.DepolarizingChannel(p, wires=1)
    qml.RY(theta/2, wires=0)
    qml.DepolarizingChannel(p, wires=0)

def adj_circuit(theta,p):
    qml.adjoint(qml.RY)(theta/2, wires=0)
    qml.DepolarizingChannel(p, wires=0)
    qml.CNOT( wires=[0,1])
    qml.DepolarizingChannel(p, wires=1)
    qml.adjoint(qml.RX)(theta, wires=0)
    qml.DepolarizingChannel(p, wires=0)

@qml.qnode(dev)
def circuit_(theta):
    circuit_real(theta)
    return qml.expval(qml.PauliZ(0))


@qml.qnode(dev)
def scale_circuit(theta,scale_factors,p):
    for i in range(scale_factors):
        circuit(theta,p)
        adj_circuit(theta,p)
    circuit(theta,p)
    return qml.expval(qml.PauliZ(0))
# 에러 완화를 위한 서킷 실행 함수

def execute_circuit(theta, scale_factor,p):
    # 스케일 팩터에 따라 회로의 게이트 파라미터를 조정
    noisy_result = scale_circuit(theta,scale_factor,p)
    return noisy_result

# ZNE를 사용하여 에러 완화 실행
def mitigate_error(theta, scale_factors,p):
    results = [execute_circuit(theta, s,p) for s in scale_factors]
    # 선형 외삽을 사용하여 제로 노이즈 추정
    fit = np.polyfit(scale_factors, results, 1)
    zne_result = fit[1]-fit[0]*1/2
    return zne_result,results

# 파라미터와 스케일 팩터 설정
theta = np.pi / 2
scale_factors = [0, 1, 2,3]

# 에러 완화 결과
mitigated_result,results = mitigate_error(theta, scale_factors,0.1)
real_result = circuit_(theta)
print("에러 완화 결과:", mitigated_result)
print("실제 결과:", real_result)

에러 완화 결과: 0.0
실제 결과: 0.0


In [401]:
import pennylane as qml
from pennylane import numpy as np

# 양자 장치 설정
dev = qml.device('default.mixed', wires=2)


@qml.qnode(dev)
def random_circuit(params):
    """임의의 게이트를 포함하는 기본 회로"""

    for i in range(10):
        qml.RX(params[i*3+0], wires=0)
        qml.RY(params[i*3+1], wires=1)
        qml.CNOT(wires=[0, 1])
        qml.RZ(params[i*3+2], wires=0)
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))


In [406]:
import pennylane as qml
from pennylane import numpy as np



def extra_polation(circ,theta,noise_factor=[0,1,2,3],p1=0.01,p2=0.02):
    
    real_vaule = circ(theta)
    dev = circ.device
    ops = circ.qtape.operations
    ops_inv = ops[::-1]


    meas = circ.qtape.measurements[0]
    def noise_circ():
        for op in ops:
            eval(f'qml.{op}')
            if len(op.wires)>1:
                for wire in op.wires:
                    qml.DepolarizingChannel(p1, wires=wire)
                    qml.BitFlip(p2, wires=wire)
            else:
                qml.DepolarizingChannel(p1, wires=op.wires) 
                qml.BitFlip(p2, wires=op.wires)
    def noise_circ_inv():
        for op in ops_inv:
            eval(f'qml.adjoint(qml.{op})')
            if len(op.wires)>1:
                for wire in op.wires:
                    qml.DepolarizingChannel(p1, wires=wire)
                    qml.BitFlip(p2, wires=wire)
            else:
                qml.DepolarizingChannel(p1, wires=op.wires) 
                qml.BitFlip(p2, wires=op.wires)
    
    @qml.qnode(dev)
    def extra_polation(factor):
        Z = qml.PauliZ
        Y = qml.PauliY
        X = qml.PauliX
        for i in range(factor):
            noise_circ()
            noise_circ_inv()
        noise_circ()
        return eval(f"qml.{meas}")
    




    res = []
    for factor in noise_factor:
        res.append(extra_polation(factor))
    return res,real_vaule

In [407]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
# 데이터셋 생성 클래스
class QuantumData(Dataset):
    def __init__(self, size=500):
        self.data = []
        self.labels = []
        for _ in tqdm(range(size)):
            theta = [np.random.uniform(0, 2 * np.pi) for i in range(30)]
            noise_factor,result = extra_polation(random_circuit,theta,noise_factor=[0,1,2,3],p1=0.01,p2=0.02)
            self.data.append(np.array(noise_factor))
            self.labels.append(result)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

# 데이터셋 및 데이터 로더 생성
dataset = QuantumData()
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

  6%|▌         | 28/500 [00:27<07:27,  1.06it/s]

In [145]:
import torch.nn as nn

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 1)
        self.relu = nn.GELU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return torch.squeeze(self.fc3(x))

model = RegressionModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [146]:
from torch.utils.data import random_split, DataLoader
# 데이터셋 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 데이터 로더
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# 모델, 손실 함수, 옵티마이저 초기화
model = RegressionModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 훈련 및 검증 루프
num_epochs = 5000
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 1, Train Loss: 0.2640, Val Loss: 0.1800
Epoch 2, Train Loss: 0.2247, Val Loss: 0.1589
Epoch 3, Train Loss: 0.2003, Val Loss: 0.1387
Epoch 4, Train Loss: 0.1673, Val Loss: 0.1171
Epoch 5, Train Loss: 0.1440, Val Loss: 0.0938
Epoch 6, Train Loss: 0.1115, Val Loss: 0.0679
Epoch 7, Train Loss: 0.0773, Val Loss: 0.0425
Epoch 8, Train Loss: 0.0423, Val Loss: 0.0211
Epoch 9, Train Loss: 0.0209, Val Loss: 0.0074
Epoch 10, Train Loss: 0.0064, Val Loss: 0.0020
Epoch 11, Train Loss: 0.0024, Val Loss: 0.0020
Epoch 12, Train Loss: 0.0025, Val Loss: 0.0026
Epoch 13, Train Loss: 0.0026, Val Loss: 0.0019
Epoch 14, Train Loss: 0.0017, Val Loss: 0.0011
Epoch 15, Train Loss: 0.0010, Val Loss: 0.0007
Epoch 16, Train Loss: 0.0008, Val Loss: 0.0005
Epoch 17, Train Loss: 0.0007, Val Loss: 0.0004
Epoch 18, Train Loss: 0.0005, Val Loss: 0.0003
Epoch 19, Train Loss: 0.0004, Val Loss: 0.0003
Epoch 20, Train Loss: 0.0003, Val Loss: 0.0002
Epoch 21, Train Loss: 0.0002, Val Loss: 0.0001
Epoch 22, Train Loss: 

KeyboardInterrupt: 

In [158]:
model(inputs[5]),inputs[5],labels[5]

(tensor(0.2495, grad_fn=<SqueezeBackward0>),
 tensor([0.1854, 0.1046, 0.0590, 0.0333]),
 tensor(0.2468))